Este notebook asume que se está ejecutando en Google Colab, y que el dataset `LLM-Redial` disponible en https://drive.google.com/drive/folders/1TIP4PFm9z0C4R4--KnHoWuiB1uK-dv5m se encuentra descargado en el drive del usuario.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import shutil
import os

source_path = '/content/gdrive/MyDrive/Proyecto LLMonkeys/Tools.py'
destination_path = '/content/Tools.py'

shutil.copy(source_path, destination_path)

'/content/Tools.py'

In [ ]:
import zipfile

zip_path = '/content/gdrive/MyDrive/Proyecto LLMonkeys/LLM_Redial.zip'
extract_path = '/content/LLM_Redial'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import dill
try:
    dill.load_session('/content/gdrive/MyDrive/Proyecto LLMonkeys/Falcon_notebook_env.db')
except FileNotFoundError:
    print("El archivo no existe")
except dill.UnpicklingError:
    print("Archivo no válido")
except EOFError:
    print('EOFError')

El archivo no existe


In [ ]:
from transformers import AutoTokenizer

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import Tools as t

# Cargar datos del Dataset

path = "./LLM_Redial/Movie"

final_data_path = '{}/final_data.jsonl'.format(path)
Conversation_path = '{}/Conversation.txt'.format(path)
user_map_path = '{}/user_ids.json'.format(path)
item_map_path = '{}/item_map.json'.format(path)

final_data = t.read_jsonl(final_data_path)
user_map = t.read_json(user_map_path)
item_map = t.read_json(item_map_path)
Conversation = t.read_dialogue(Conversation_path)

**Separación de la información de los usuarios en train y test**


In [ ]:
import json
import random

path = './LLM_Redial/Movie/final_data.jsonl'

# Cada entrada de final_data.jsonl se ve así
#{
#  "A30Q8X8B1S3GGT": {
#    "history_interaction": [...],
#    "user_might_like": [...],
#    "Conversation": [...]
#  }
#}


with open(path, 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Dividimos los dialogos en 80% & 20% para training y testing.
n_conversations = 10089
train_len = n_conversations * 0.8
test_len = n_conversations * 0.2

train_conv = []
test_conv = []
used_users = []

aux = []

# Integramos el 20% de usuarios a Test
while len(aux) < test_len:
    try:
        convs = []
        # "La semilla random.seed(42) se reinicia dentro del bucle, por lo que no tiene efecto real. Debería estar fuera del bucle para resultados reproducibles."
        random.seed(42)

        # Usando used_users aseguramos que un mismo usuario no aparezca en Train y Test al mismo tiempo
        user_id = random.choice(list(set(user_map.keys())^set(used_users)))
        used_users.append(user_id)

        # Guardamos la info del usuario y sus dialogos
        user_data = next((item[user_id] for item in data if user_id in item), None)
        user_conversations = user_data.get("Conversation", [])

        for i in range(len(user_conversations)):
            selected_conversation = user_conversations[i]
            conversation_details = list(selected_conversation.values())[0]
            conversation_id = conversation_details["conversation_id"]
            if conversation_id != 10088:
                conversation = Conversation[Conversation.index(f"{conversation_id}\n"):Conversation.index(f"{conversation_id+1}\n")]
            else:
                conversation = Conversation[Conversation.index(f"{conversation_id}\n"):]

            convs.append(conversation)
            aux.append(conversation)
        test_conv.append([user_id,convs])
    except ValueError:
        print("ValueError: ", user_id, f"{conversation_id}\n")

# Integramos el resto de los usuarios y su info a Training
for user_id in list(set(user_map.keys()) ^ set(used_users)):
    try:
        convs = []
        user_data = next((item[user_id] for item in data if user_id in item), None)
        user_conversations = user_data.get("Conversation", [])
        for i in range(len(user_conversations)):
            selected_conversation = user_conversations[i]
            conversation_details = list(selected_conversation.values())[0]
            conversation_id = conversation_details["conversation_id"]
            if conversation_id != 10088:
                conversation = Conversation[Conversation.index(f"{conversation_id}\n"):Conversation.index(f"{conversation_id+1}\n")]
            else:
                conversation = Conversation[Conversation.index(f"{conversation_id}\n"):]

            convs.append(conversation)
        train_conv.append([user_id,convs])
    except ValueError:
        print("ValueError: ", user_id, f"{conversation_id}\n")

print("Número de usuarios en conjunto de TEST:", len(test_conv))
print("Número de usuarios en conjunto de TRAIN:", len(train_conv))
print("Total de usuarios en test + train:", len(test_conv) + len(train_conv))
print("Total de conversaciones originales esperadas:", n_conversations)

Número de usuarios en conjunto de TEST: 642
Número de usuarios en conjunto de TRAIN: 2489
Total de usuarios en test + train: 3131
Total de conversaciones originales esperadas: 10089


In [ ]:
# Darle formato de lista a las respuestas
def format_ans(ans,n,i):
    try:
        idx = [ans.index(f"{i}. ") for i in range(1, n+1)]
        idx_c = idx[n-1] + ans[idx[n-1]:].index(chr(34))
        idx_end = idx_c + ans[idx_c+1:].index(chr(34))

        answer_list = [ans[idx[i]+3:idx[i+1]-1] for i in range(n-1)]
        answer_list.append(ans[idx[n-1]+3:idx_end+2])
        answer_str = ", ".join(answer_list)
        answer = f"{i}. {answer_str}\n"
        i+=1

    except Exception as e:
        answer = e

    finally:
        return answer, i

# Output ex. 1. The Matrix, Inception, Interstellar, Arrival

### Selección de diálogo a utilizar

In [ ]:
num_test_items = 10

In [ ]:
# Generamos ejemplos de conversaciones al azar

rand_conversations = []

for n in range(num_test_items):
  random.seed(n)
  rand_user = random.choice(test_conv)
  user_id = rand_user[0]
  user_conversation  = random.choice(rand_user[1])

  user_data = next((item[user_id] for item in data if user_id in item), None)
  convs = user_data.get("Conversation", [])

  for i in range(len(rand_user[1])):
    if user_conversation == rand_user[1][i]:
      rand_user_conv_id = i

  dialog = "\n\n".join(user_conversation.split("\n\n")[1:4])
  dialog_id = user_conversation.split("\n\n")[0]
  dialog_ground_truth = list(convs[rand_user_conv_id].values())[0]["rec_item"]

  rand_conversations.append([
      rand_user_conv_id, # 0: Índice de conversación
      dialog, # 1: Texto del diálogo parcial
      user_data, # 2: Info estructurada del usuario
      dialog_id, # 3: ID de conversación
      dialog_ground_truth # 4: Item recomendado como verdad
      ])

print(rand_conversations[0][4]) # dialog_ground_truth

['B00A7ZH8GM']


In [ ]:
# Appendeamos a rand_conversations las interacciones del usuario para tenerlas a mano

for i in range(num_test_items):
  user_data = rand_conversations[i][2]
  rand_user_interactions = user_data.get("history_interactions", [])

  rand_user_interactions = [item_map[m] for m in rand_user_interactions]
  rand_conversations[i].append(rand_user_interactions)

In [ ]:
# Preparamos ejemplos representativos aleatorios para entrenar

random.seed(42)
train_users = random.sample(train_conv, 5)

for u in train_users:
  user_data = next((item[u[0]] for item in data if u[0] in item), None)
  user_interactions = user_data.get("history_interaction", [])
  user_interactions = [item_map[m] for m in user_interactions]

  conversation = min(u[1], key=len)
  conversation[conversation.index("User:"):-2]
  convs = user_data.get("Conversation", [])
  for c in convs:
    user_likes = list(c.values())[0]["user_likes"]
    user_dislikes = list(c.values())[0]["user_dislikes"]
    recs= list(c.values())[0]["rec_item"]

    user_likes = [item_map[m] for m in user_likes]
    user_dislikes = [item_map[m] for m in user_dislikes]
    recs = [item_map[m] for m in recs]

## Falcon 1.1B


### Zero-Shot sin interacciones históricas:

In [ ]:
from transformers import pipeline
import torch

outputs_z_n = []

for n in range(num_test_items):
  k = 12
  outputs = []

  pipe = pipeline(
      "text-generation",
      model=model_name,
      device_map="auto",
      torch_dtype=torch.float16,
  )

  ctx = (
      "Me das una lista de 10 peliculas que recomiendes? Solamente dame la lista de las 10 peliculas enumeradas del 1 al 10, con sus nombres en ingles, y sin repetir la misma pelicula en la lista. No digas nada mas."
  )


  messages = [{"role": "user", "content": ctx}]

  prompt = ctx
  inputs = pipe.tokenizer(prompt, return_tensors="pt").to("cuda")

  with torch.no_grad():
      for i in range(k):
          output_ids = pipe.model.generate(
          **inputs,
          max_new_tokens=200,
          do_sample=True,
          temperature=0.4,
          top_k=40,
          top_p=0.9,
      )
          decoded = pipe.tokenizer.decode(output_ids[0], skip_special_tokens=True)
          outputs.append(decoded)
  outputs_z_n.append(outputs)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation

## Evaluación

In [ ]:
user_conversation_id = int(user_conversation[:user_conversation.index("User:")-2])
user_data = next((item[rand_user[0]] for item in data if rand_user[0] in item), None)
convs = user_data.get("Conversation", [])
ground_truth = [item_map[m] for m in list(convs[rand_user_conv_id].values())[0]["rec_item"]]


print(user_conversation_id)
print(ground_truth)

1968
['Rabbit-Proof Fence']


In [ ]:
for c in rand_conversations:
  print(c[4])
  ground_truth = [item_map[m] for m in c[4]]
  print(ground_truth)

['B00A7ZH8GM']
['Good Day To Die Hard, A']
['B00005JKWI']
['Blade 2 VHS']
['6301964306']
['Alexander the Great VHS']
['B00027SIUK']
['Dawn of the Dead']
['B01A9LM9B2']
['Disturbia']
['0767834739']
['Resident Evil: Apocalypse']
['B0001CNRNO']
['The Monster Legacy Gift Set: (Frankenstein / Dracula / The Wolf Man)']
['6305842000']
['The Bat']
['6300185117']
['Destry Rides Again VHS']
['B00005JLD4']
['Rabbit-Proof Fence']


In [ ]:
import numpy as np
import re
from sklearn.metrics import ndcg_score

# Funciones generadas por DeepSeek
def recall_at_k(generated_recommendations, ground_truth, k=10):
    # Tomar las primeras K recomendaciones generadas
    top_k = generated_recommendations[:k]

    # Convertir a conjuntos para comparación
    generated_set = set(top_k)
    true_set = set(ground_truth)

    # Calcular intersección (recomendaciones correctas en top K)
    hits = len(generated_set.intersection(true_set))

    # Evitar división por cero
    return hits / len(true_set) if len(true_set) > 0 else 0.0

def ndcg_at_k(generated_recommendations, ground_truth, k=10):
    # Crear una lista binaria de relevancia (1 si está en ground truth, 0 si no)
    relevance = [1 if item in ground_truth else 0 for item in generated_recommendations[:k]]

    # Crear el "ideal ranking" (todas las relevantes primero)
    ideal_relevance = sorted(relevance, reverse=True)

    # Calcular NDCG
    return ndcg_score([relevance], [ideal_relevance])

pattern = r"\d+\.\s(.*?)\s\(\d{4}\)"

### Recall@5-10 y NDCG@5-10:

In [ ]:
z_n_rec_lists = []
i = 1
for outputs in outputs_z_n:
    rec_lists = []
    for out in outputs:
        lines = out.splitlines()
        movie_lines = []
        for line in lines:
            if re.match(r'^10\.', line):
                movie_lines.append(line.strip())
                break
            if re.match(r'^\d+\.', line):
                movie_lines.append(line.strip())

        trimmed_output = "\n".join(movie_lines)
        z_n_rec_lists.append(trimmed_output)

z_n_clean_lists = []

for block in z_n_rec_lists:
    lines = block.splitlines()
    cleaned = []

    for line in lines:
        line = re.sub(r'^\d+\.\s*', '', line)
        line = re.sub(r'\s*\(\d{4}\)', '', line)
        line = line.strip()
        cleaned.append(line)

    cleaned = list(dict.fromkeys(cleaned))

    z_n_clean_lists.append(cleaned)

print(z_n_clean_lists)

[['The Girl with the Dragon Tattoo', 'The Girl Who Played with Fire', 'The Girl Who Kicked the Hornet’s Nest'], ['The Godfather', 'The Godfather Part II', 'The Godfather Part III', 'The Godfather: Part II', 'The Godfather: Part III'], ['The Godfather', 'The Godfather Part II', 'The Godfather: Part III', 'The Godfather: Part IV', 'The Godfather: Part V', 'The Godfather: Part VI', 'The Godfather: Part VII', 'The Godfather: Part VIII', 'The Godfather: Part IX', 'The Godfather: Part X'], ['The Lord of the Rings', 'The Lord of the Rings: The Return of the King', 'The Lord of the Rings: The Fellowship of the Ring', 'The Lord of the Rings: The Two Towers', 'The Lord of the Rings: The Return of the King (Extended Edition)', 'The Lord of the Rings: The Fellowship of the Ring (Extended Edition)', 'The Lord of the Rings: The Return of the King (Extended Edition) (Spanish Edition)'], ['The Princess Bride'], ['The Girl on the Train'], ['The Matrix', 'The Matrix Reloaded', 'The Matrix Revolutions', 

In [ ]:
print("Ground Truth (Global por índole del experimento):", ground_truth)

total_recall_5 = 0.0
total_ndcg_5 = 0.0
total_recall_10 = 0.0
total_ndcg_10 = 0.0

valid_count = 0

for i, predictions in enumerate(z_n_clean_lists):
    if len(predictions) < 5:
        print(f"[{i}] Lista con menos de 5 elementos. Saltando.")
        continue

    recall_5 = recall_at_k(predictions, ground_truth, k=5)
    ndcg_5 = ndcg_at_k(predictions, ground_truth, k=5)
    recall_10 = recall_at_k(predictions, ground_truth, k=10)
    ndcg_10 = ndcg_at_k(predictions, ground_truth, k=10)

    print(f"[{i}] Recall@5: {recall_5:.4f}, NDCG@5: {ndcg_5:.4f}, Recall@10: {recall_10:.4f}, NDCG@10: {ndcg_10:.4f}")

    total_recall_5 += recall_5
    total_ndcg_5 += ndcg_5
    total_recall_10 += recall_10
    total_ndcg_10 += ndcg_10
    valid_count += 1

if valid_count > 0:
    print("\n=== Promedios Globales ===")
    print(f"Recall@5: {total_recall_5 / valid_count:.4f}")
    print(f"NDCG@5: {total_ndcg_5 / valid_count:.4f}")
    print(f"Recall@10: {total_recall_10 / valid_count:.4f}")
    print(f"NDCG@10: {total_ndcg_10 / valid_count:.4f}")
else:
    print("No se encontraron listas válidas para evaluación.")

Ground Truth (Global por índole del experimento): ['Rabbit-Proof Fence']
[0] Lista con menos de 5 elementos. Saltando.
[1] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[2] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[3] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[4] Lista con menos de 5 elementos. Saltando.
[5] Lista con menos de 5 elementos. Saltando.
[6] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[7] Lista con menos de 5 elementos. Saltando.
[8] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[9] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[10] Lista con menos de 5 elementos. Saltando.
[11] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[12] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[13] Recall@5: 0.0000, NDCG@5: 0.0000, Recall@10: 0.0000, NDCG@10: 0.0000
[14] Lista con menos de 5 elementos.

In [ ]:
path = '/content/gdrive/MyDrive/Proyecto LLMonkeys/Falcon_notebook_env.db'
with open(path, 'wb') as f:
    dill.dump_session(f)